In [ ]:
import os
import time
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
import pandas as pd
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut, cross_validate
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, Normalizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.ensemble import (
    AdaBoostRegressor,
    RandomForestRegressor,
    ExtraTreesRegressor,
)
from sklearn.linear_model import ElasticNetCV, SGDRegressor, RidgeCV, LassoLarsCV
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator
from xgboost import XGBRegressor
from sklearn.metrics import (
    explained_variance_score,
    max_error,
    mean_absolute_error,
    mean_squared_error,
)
from sklearn.metrics import (
    mean_squared_log_error,
    median_absolute_error,
    r2_score,
    accuracy_score,
)

In [ ]:
data_dir = "" # Replace with the path to the PyFeat data directory